In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Lenovo\\Documents\\Work\\Learning\\ML_DataScience\\Workspace\\Visual_Studio\\Chicken-Disease-Classification-DL\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Lenovo\\Documents\\Work\\Learning\\ML_DataScience\\Workspace\\Visual_Studio\\Chicken-Disease-Classification-DL'

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [21]:
class ConfigurationManager:
    def __init__(
            self,
            config_path = CONFIG_FILE_PATH,
            params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
            config = self.config.prepare_callbacks

            model_check_point_dir = os.path.dirname(config.checkpoint_model_filepath)
            create_directories([Path(config.tensorboard_root_log_dir), Path(model_check_point_dir)])

            prepare_callback_config = PrepareCallbackConfig(
                root_dir=config.root_dir,
                tensorboard_root_log_dir=config.tensorboard_root_log_dir,
                checkpoint_model_filepath=config.checkpoint_model_filepath
            )

            return prepare_callback_config


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        training_data = os.path.join(self.config.data_ingestion.unzip_dir, 'Chicken-fecal-images')

        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir= Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=training_data,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [13]:
import time

In [14]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config

    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime('%Y-%m-%d-%H-%M-%S')
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}")

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_check_point_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    

    def get_tb_checkpoint_callbacks(self):

        return [
            self._create_tb_callbacks,
            self._create_check_point_callbacks
        ]



In [15]:
import os
from pathlib import Path
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [16]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [23]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_checkpoint_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-02-28 00:41:55,730: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-28 00:41:55,732: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-28 00:41:55,734: INFO: common: created directory at: artifacts]
[2025-02-28 00:41:55,734: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-02-28 00:41:55,735: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-02-28 00:41:55,736: INFO: common: created directory at: artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


c:\Users\Lenovo\anaconda3\envs\chicken\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5048 - loss: 10.4764

c:\Users\Lenovo\anaconda3\envs\chicken\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.5051 - loss: 10.6046 - val_accuracy: 0.3906 - val_loss: 7.7894
Epoch 2/5
 1/19 ━━━━━━━━━━━━━━━━━━━━ 53s 3s/step - accuracy: 0.6250 - loss: 3.5123

c:\Users\Lenovo\anaconda3\envs\chicken\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


19/19 ━━━━━━━━━━━━━━━━━━━━ 14s 585ms/step - accuracy: 0.6250 - loss: 3.5123 - val_accuracy: 0.6094 - val_loss: 7.3542
Epoch 3/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.5853 - loss: 9.5521 - val_accuracy: 0.5469 - val_loss: 6.6299
Epoch 4/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 442ms/step - accuracy: 0.8125 - loss: 1.2044 - val_accuracy: 0.8906 - val_loss: 0.8549
Epoch 5/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.8945 - loss: 1.0270 - val_accuracy: 0.8906 - val_loss: 0.8385
